In [3]:
import pandas as pd
import pandasdmx as sdmx
import importlib.util
import sys

# Carga las funciones de 01_API_access 
module_name = "api_access"
file_path = "01_API_access.py"
spec = importlib.util.spec_from_file_location(module_name, file_path)
api_access = importlib.util.module_from_spec(spec)
sys.modules[module_name] = api_access
spec.loader.exec_module(api_access)

In [ ]:
# Indicador a testear
selected_ind = "HVA_PED_ART_CVG"


In [4]:
indicators = api_access.get_list_of_unicef_indicators()
indicators_sdg = api_access.get_list_of_unicef_indicators(parent_database='SDG_PROG_ASSESSMENT')
cl_ref_areas = api_access.get_county_codes()

selected_ind_data = indicators.loc[selected_ind]
selected_ind_code = selected_ind_data.name
selected_ind_parent = selected_ind_data['parent']

df = api_access.query_indicator(selected_ind_code, selected_ind_parent)
ds = api_access.format_dataframe(df, cl_ref_areas)
print("Este indicador anda bien!!")
ds

,REF_AREA,COUNTRY,INDICATOR,TIME_PERIOD,SEX,value
0,AFG,Afghanistan,HVA_PED_ART_CVG,2010,_T,1
1,AFG,Afghanistan,HVA_PED_ART_CVG,2011,_T,3
2,AFG,Afghanistan,HVA_PED_ART_CVG,2012,_T,3
3,AFG,Afghanistan,HVA_PED_ART_CVG,2013,_T,6
4,AFG,Afghanistan,HVA_PED_ART_CVG,2014,_T,8
...,...,...,...,...,...,...
1488,SDGRC_ECA_CA,(SDGRC) Central Africa,HVA_PED_ART_CVG,2023,_T,31
1489,SDGRC_ECA_EA,(SDGRC) Eastern Africa,HVA_PED_ART_CVG,2023,_T,62
1490,SDGRC_ECA_NA,(SDGRC) North Africa,HVA_PED_ART_CVG,2023,_T,39
1491,SDGRC_ECA_SA,(SDGRC) Southern Africa,HVA_PED_ART_CVG,2023,_T,65


In [8]:
### Apertura de la función drops_irrelevant_index_levels() sin las cosas irrelevantes
# Replace COUNTRY with REF_AREA (SDMX standard)
df.index.names = ["REF_AREA" if idx == "COUNTRY" else idx for idx in df.index.names]

indicador = df.index.get_level_values("INDICATOR").unique().values[0]
indexes = df.index.names
irrelevant_indexes = [
    idx
    for idx in indexes
    if idx not in ["REF_AREA", "INDICATOR", "TIME_PERIOD", "SEX", "DATA_SOURCE"]
]

for idx in irrelevant_indexes:
    totals_mask = df.index.get_level_values(idx) == "_T"
    if totals_mask.sum() > 0:
        df = df[totals_mask].droplevel(idx)

value
REF_AREA   INDICATOR       AGE    SEX DATA_SOURCE TIME_PERIOD       
AFG        HVA_PED_ART_CVG Y0T14  _T  127         2010             1
                                                  2011             3
                                                  2012             3
                                                  2013             6
                                                  2014             8
...                                                              ...
UNICEF_WCA HVA_PED_ART_CVG Y10T19 M   hiv_src_115 2017         22.20
                                                  2018         25.70
                                                  2019         30.80
                                                  2020         38.60
                                                  2021         47.60

[6482 rows x 1 columns]

In [12]:
print(df.index.get_level_values("AGE").value_counts())

AGE
Y10T19    4989
Y0T14     1493
Name: count, dtype: int64


In [6]:
df

,,,,,,,,value
REF_AREA,INDICATOR,AGE,SEX,WEALTH_QUINTILE,RESIDENCE,DATA_SOURCE,TIME_PERIOD,
